In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
import sys
sys.path.append("/workspaces/remember-me/back/src")


In [3]:
from remember.batch.generate_event_memories import generate_event_memories, save_event_memories, load_event_memories
from remember.batch.generate_day_memory import generate_day_memory, load_day_memory, save_day_memory

In [ ]:
event_memories = generate_event_memories(event = "Les vacances à Rome", 
                  name = "Monique",
                  age = 75,
                  occupation = "Ancienne Assistante sociale",
                  nb_memories = 15) 

event_memories.model_dump()

In [7]:
save_event_memories(event_memories,"event_memories.json")

In [4]:
event_memories = load_event_memories("event_memories.json")

In [5]:
day_memory = generate_day_memory(name = "Monique",
                  age = 75,
                  occupation = "Ancienne Assistante sociale",
                  memories = event_memories)

In [7]:
save_day_memory(day_memory,"day_memory.json")

In [ ]:
day_memory = load_day_memory("day_memory.json")

In [8]:
day_memory

DayMemory(title='Les vacances à Rome', summary="Monique, te souviens-tu de nos vacances à Rome ? La ville éternelle nous a accueillis avec sa chaleur et ses merveilles. Nous avons flâné dans les ruelles pavées, émerveillés par l'histoire qui se dévoilait à chaque coin de rue. Le Colisée, majestueux sous le soleil, t'a particulièrement marquée. Tu nous racontais des anecdotes sur les gladiateurs comme si tu y étais. La Fontaine de Trevi, avec ses eaux scintillantes, a vu nos vœux les plus chers. Et que dire de la Piazza Navona, où tu riais aux éclats avec les artistes de rue ? Un mime t'a même invitée à danser, c'était si drôle et léger. Rome, c'était aussi nos soirées à Trastevere, à déguster des plats typiques et à écouter de la musique italienne. Tu chantais et riais comme si tu avais 20 ans. Ces moments de complicité et de joie resteront à jamais gravés dans nos cœurs. Rome, c'était notre aventure, notre bonheur partagé.")

In [9]:
from langchain.chat_models import init_chat_model
from elevenlabs.client import ElevenLabs
from elevenlabs import play, save
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import display, Markdown
from typing import List
from enum import Enum
import os

from pydantic import BaseModel, Field
import json
from typing import List

In [10]:
#model = init_chat_model("ministral-8b-latest", model_provider="mistralai", temperature=0)
model = init_chat_model("mistral-large-latest", model_provider="mistralai", temperature=0.2)
#model = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.3, max_retries=3)

In [11]:
class Period(str, Enum):
    """Enum for periods of life."""
    CHILDHOOD = "childhood"
    ADOLESCENCE = "adolescence"
    YOUNG_ADULTHOOD = "young_adulthood"
    ADULTHOOD = "adulthood"
    OLD_AGE = "old_age"

class Person(BaseModel):
    id: int = Field(description="The unique identifier for the person")
    name: str = Field(description="The name of the person")
    relationship: str = Field(description="The relationship of the person to the user")
    age: int = Field(description="The age of the person")
    occupation: str = Field(description="The occupation of the person")
    hobbies: List[str] = Field(default_factory=list, description="The hobbies of the person")
    description: str = Field(description="A description of the person")

    def __str__(self):
        return json.dumps(self.dict(), indent=2)

class Memory(BaseModel):
    people: List[int] = Field(min_items=1, decription="A list of IDs of people involved in the memory")
    place: List[str] = Field(min_items=1, default_factory=list, description="A list of places involved in the memory")
    period: List[str] = Field(min_items=1, default_factory=list)
    emotion: List[str] = Field(min_items=1, default_factory=list)
    description: str = Field(description="A description of the memory")

    def __str__(self):
        return json.dumps(self.dict(), indent=2)
    
class LifeSummary(BaseModel):
    name: str = Field(description="The name of the user")
    age: int = Field(description="The age of the user")
    occupation: str = Field(description="The occupation of the user")
    acquaintances: List[Person] = Field(min_items=5, default_factory=list, description="A list of acquaintances of the user (friend, colleague,  family)")
    memories: List[Memory] = Field(default_factory=list)

    def __str__(self):
        return json.dumps(self.dict(), indent=2)

# Génération des souvenirs pour la vie

In [12]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage("Tu est un chatbot qui génère des données de synthèse sur les souvenirs sur toute la vie d'une personne agée."),
        ("user", """Invente cinq souvenirs. pour une personne dont voici la description :
            Nom : {name}
            Age : {age}
            Profession : {occupation} 
         Rédige les souvenirs à la première personne,
         en utilisant le vocabulaire et les émotions d'une personne âgée.
         Chaque souvenir doit être spécifique et détaillé, en incluant la référence (id) des personnes impliquées, les lieux, la période de la vie et les émotions ressenties."""),
    ]
)
chain = prompt_template | model.with_structured_output(LifeSummary)

In [13]:
life_summary = chain.invoke({"name": "Monique", "age": "75", "occupation": "Ancienne Assistante sociale"})

In [15]:
life_summary.model_dump()

{'name': 'Monique',
 'age': 75,
 'occupation': 'Ancienne Assistante sociale',
 'acquaintances': [{'id': 1,
   'name': 'Pierre',
   'relationship': 'Mari',
   'age': 78,
   'occupation': 'Retraité',
   'hobbies': ['Jardinage', 'Lecture'],
   'description': 'Mon mari depuis 50 ans, un homme patient et aimant.'},
  {'id': 2,
   'name': 'Sophie',
   'relationship': 'Fille',
   'age': 45,
   'occupation': 'Professeur',
   'hobbies': ['Peinture', 'Randonnée'],
   'description': 'Ma fille aînée, une femme indépendante et passionnée par son métier.'},
  {'id': 3,
   'name': 'Jean',
   'relationship': 'Fils',
   'age': 42,
   'occupation': 'Ingénieur',
   'hobbies': ['Football', 'Cuisine'],
   'description': 'Mon fils cadet, un homme dévoué à sa famille et à son travail.'},
  {'id': 4,
   'name': 'Marie',
   'relationship': "Amie d'enfance",
   'age': 75,
   'occupation': 'Retraitée',
   'hobbies': ['Tricot', 'Voyages'],
   'description': "Mon amie d'enfance, toujours présente dans les moments 

# Génération de souvenirs pour un évènement particulier

In [8]:
class Person(BaseModel):
    name: str = Field(description="The name of the person")
    relationship: str = Field(description="The relationship of the person to the user")
    age: int = Field(description="The age of the person")
    occupation: str = Field(description="The occupation of the person")
    hobbies: List[str] = Field(default_factory=list, description="The hobbies of the person")
    description: str = Field(description="A description of the person")

    def __str__(self):
        return json.dumps(self.dict(), indent=2)

class Memory(BaseModel):
    person: Person = Field(decription="The writer of the memory")
    place: List[str] = Field(default_factory=list, description="A list of places involved in the memory")
    emotion: List[str] = Field(default_factory=list)
    description: str = Field(description="A description of the memory")

    def __str__(self):
        return json.dumps(self.dict(), indent=2)
    
class EnventMemories(BaseModel):
    """Memories collection about a single event with different perspectives."""
    event: str = Field(description="The specific event being remembered")
    memories: List[Memory] = Field(default_factory=list)

    def __str__(self):
        return json.dumps(self.dict(), indent=2)

In [9]:
prompt_template = ChatPromptTemplate(
    [
        ("system", """Tu es un générateur de souvenirs narratifs, capable de simuler différents styles d’écriture, 
                      tons émotionnels et points de vue. Tu adoptes le rôle de plusieurs proches d'une personne agée pour raconter leurs souvenirs de leur point de vue pour un même évènement."""),
        ("human", """Invente {nb_memories} souvenirs riches, spécifiques et émotionnels racontés par différents proches de la personne agée et liés à un même évènement ‘‘‘{event}‘‘‘. 
                      Chaque souvenir doit être rédigé à la première personne, dans un style adapté à la personnalité du narrateur.

                      **Contexte de la personne agée** :
                        - Nom : {name}
                        - Âge : {age}
                        - Ancienne profession : {occupation}
                        - Elle a une mémoire défficiente, et ses proches souhaitent lui faire revivre des moments heureux à travers ces souvenirs.
                     
                     Rédige les souvenirs à la première personne au sujet de l'évènement : ‘‘‘{event}‘‘‘. 
                     Chaque souvenir est rédigé par un proche distinct (par exemple amis, enfants, petits-enfants, voisins, collègues retraités, etc.).
                     Chaque souvenir doit être spécifique et détaillées et en intégrant les émotions ressenties. Utilise des anecdotes.
                     """
        )
    ]
)
chain = prompt_template | model.with_structured_output(EnventMemories)

In [10]:
event_memories = chain.invoke({"name": "Monique",
"age": "75",
"occupation": "Ancienne Assistante sociale",
"event": "Mariage de Thibault, le petit fils de Monique",
"nb_memories": 15})

In [11]:
event_memories.model_dump()

{'event': 'Mariage de Thibault, le petit fils de Monique',
 'memories': [{'person': {'name': 'Marie',
    'relationship': 'Fille de Monique',
    'age': 50,
    'occupation': 'Professeur',
    'hobbies': [],
    'description': 'Une femme attentionnée et aimante, toujours prête à aider les autres.'},
   'place': ['Église Saint-Pierre', 'Jardin des Roses'],
   'emotion': ['Joie', 'Fierté', 'Nostalgie'],
   'description': "Le jour du mariage de Thibault, je me souviens de l'émotion palpable dans l'église. Monique, ma mère, était resplendissante dans sa robe bleu pastel. Elle tenait fièrement le bras de Thibault alors qu'ils remontaient l'allée. Les larmes de joie coulaient sur ses joues, et je pouvais voir dans ses yeux tout l'amour et la fierté qu'elle ressentait pour son petit-fils. Pendant la réception au Jardin des Roses, elle n'a pas cessé de sourire, partageant des anecdotes sur Thibault quand il était petit. C'était un moment de bonheur pur, et je suis si heureuse d'avoir pu le par

In [12]:
with open("event_memories.json", "w", encoding="utf-8") as file:
    file.write(event_memories.model_dump_json(indent=2))

# Synthèse de l'histoire

In [13]:
with open("event_memories.json", "r", encoding="utf-8") as file:
    event_memories = json.load(file)

In [10]:
day_memory

DayMemory(title='Les vacances à Rome', summary="Monique, te souviens-tu de nos vacances à Rome ? La ville éternelle nous a accueillis avec sa chaleur et ses merveilles. Nous avons flâné dans les ruelles pavées, émerveillés par l'histoire qui se dévoilait à chaque coin de rue. Le Colisée, majestueux sous le soleil, t'a particulièrement marquée. Tu nous racontais des anecdotes sur les gladiateurs comme si tu y étais. La Fontaine de Trevi, avec ses eaux scintillantes, a vu nos vœux les plus chers. Et que dire de la Piazza Navona, où tu riais aux éclats avec les artistes de rue ? Un mime t'a même invitée à danser, c'était si drôle et léger. Rome, c'était aussi nos soirées à Trastevere, à déguster des plats typiques et à écouter de la musique italienne. Tu chantais et riais comme si tu avais 20 ans. Ces moments de complicité et de joie resteront à jamais gravés dans nos cœurs. Rome, c'était notre aventure, notre bonheur partagé.")

In [11]:
display(Markdown(day_memory.summary))

Monique, te souviens-tu de nos vacances à Rome ? La ville éternelle nous a accueillis avec sa chaleur et ses merveilles. Nous avons flâné dans les ruelles pavées, émerveillés par l'histoire qui se dévoilait à chaque coin de rue. Le Colisée, majestueux sous le soleil, t'a particulièrement marquée. Tu nous racontais des anecdotes sur les gladiateurs comme si tu y étais. La Fontaine de Trevi, avec ses eaux scintillantes, a vu nos vœux les plus chers. Et que dire de la Piazza Navona, où tu riais aux éclats avec les artistes de rue ? Un mime t'a même invitée à danser, c'était si drôle et léger. Rome, c'était aussi nos soirées à Trastevere, à déguster des plats typiques et à écouter de la musique italienne. Tu chantais et riais comme si tu avais 20 ans. Ces moments de complicité et de joie resteront à jamais gravés dans nos cœurs. Rome, c'était notre aventure, notre bonheur partagé.

In [12]:

# Initialize ElevenLabs client with your API key
api_key = os.getenv("ELEVENLABS_API_KEY")
client = ElevenLabs(api_key=api_key)
text = day_memory.summary
voice_id = "sSSnx1yBXjhBo6gzmfyI"

# Generate audio from text using the specified voice ID
audio = client.generate(
    text=text,
    voice=voice_id,
    model="eleven_multilingual_v2",
    output_format="mp3_44100_128",

)

output_path = "output.mp3"

# Save the generated audio to a file
save(audio, output_path)
print(f"Audio saved to {output_path}")

Audio saved to output.mp3
